In [1]:
import ee
ee.Initialize()
from iteru import *
from ipywidgets import *


In [2]:
map = Map()

In [3]:
basemaps = Dropdown(
                     options = basemaps,
                     value = None,
                     description = '',
                     description_tooltip = 'Select Basemap',
                     layout=Layout(width='200px'),
                     style = {'description_width': 'initial'}
               )

terrain = Dropdown(
                     options = terrain,
                     value = None,
                     description = '',
                     description_tooltip = 'GEE Terrain Dataset',
                     layout=Layout(width='200px'),
                     style = {'description_width': 'initial'}
             )

TypeError: 'Dropdown' object is not iterable

In [ ]:
basemaps_button = ToggleButton(
                               value = False, 
                               tooltip = 'Basemaps',
                               icon = 'map',
                               layout = Layout(width = '32px', height = '28px'),
        )
terrain_dataset_button = ToggleButton(
                               value = False, 
                               tooltip = 'GEE Terrain Dataset',
                               icon = 'server',
                               layout = Layout(width = '32px', height = '28px'),
        )
TOC = ToggleButton(
                               value = False, 
                               tooltip = 'Table of Contents',
                               icon = 'archive',
                               layout = Layout(width = '32px', height = '28px'),
       )

In [ ]:
basemap_tool = HBox([basemaps_button])
terrain_dataset_tool = HBox([terrain_dataset_button])
TOC_container = HBox([TOC])
TOC_out = Output(layout={'border': '1px solid blue'})

In [ ]:
def basemap_tool_click(change):
    if change.new:
        basemap_tool.children = [basemaps_button,basemaps]
    else:
        basemap_tool.children = [basemaps_button]
        
        
def terrain_tool_click(change):
    if change.new:
        terrain_dataset_tool.children = [terrain_dataset_button, terrain]
    else:
        terrain_dataset_tool.children = [terrain_dataset_button]
        
        
def TOC_container_click(change):
    if change.new:
        TOC_container.children = [TOC,TOC_out]
    else:
        TOC_container.children = [TOC]      

In [ ]:
TOC.observe(TOC_container_click, names = 'value')
basemaps_button.observe(basemap_tool_click, names = 'value')
terrain_dataset_button.observe(terrain_tool_click, names = 'value')

In [ ]:
basemap_tool_widget = AddWidget(widget = basemap_tool, position = 'topleft')
terrain_dataset_tool_widget = AddWidget(widget = terrain_dataset_tool, position = 'topleft')
TOC_widget = AddWidget(widget = TOC_container, position = 'topleft')

In [ ]:
map.add_control(basemap_tool_widget)
map.add_control(terrain_dataset_tool_widget)
map.add_control(TOC_widget )

In [ ]:
def map_basemaps_widgets(change):
        if change.new:
            if change.new in map.layers:
                pass
            else:
                layer = change.new
                map.add_layer(layer)
                       
                close_button = Button(  
                      value = False, 
                      tooltip = 'Remove This Layer',
                      icon = 'window-close',
                      layout = Layout(width = '32px', height = '28px'),   
                )

                layer_visibility = Checkbox(
                      value = True, 
                      indent=False,
                      layout = Layout(width = '15px')
                )
                
                layer_name = Label(
                                   value= change.new.name,
                                   layout = Layout(width = '110px', height = '28px'),
                                   tooltip = change.new.name
                              )

                opacity = FloatSlider(
                                      value = 1,
                                      min = 0,  
                                      max = 1,
                                      step = 0.1,
                                      indent=False,            
                               )
                def change_layer_visibility(show_hide):
                    change.new.visible = show_hide 
                    
                def change_layer_opacity(Opacity):
                    change.new.opacity = Opacity
                        
                def remove_layer(b):
                    map.remove_layer(layer)
                    close_button.close()
                    opacity.close()
                    layer_name.close()
                    layer_visibility.close()
                    
                opacity_slider = interactive(change_layer_opacity,Opacity = opacity)
                
                visibility_checkbox = interactive(change_layer_visibility,show_hide = layer_visibility)
                
                with TOC_out:
                    display(HBox([layer_visibility,layer_name,opacity, close_button]))
                
                
                close_button.on_click(remove_layer)
basemaps.observe(map_basemaps_widgets, names = 'value') 

In [ ]:
def add_terrain(change):
    
    if change.new:
        if ee_tilelayer(change.new) in map.layers:
            pass
        
        else:
            
            layer = ee_tilelayer(change.new, name = change.new.getInfo()['id'])
        
            map.add_layer(layer)
            
            close_button = Button(  
                                value = False, 
                                tooltip = 'Remove This Layer',
                                icon = 'window-close',
                                layout = Layout(width = '32px', height = '28px'),   
                               )

            layer_visibility = Checkbox(
                               value = True, 
                               indent=False,
                               layout = Layout(width = '15px')
                )
                
            layer_name = Label(
                                   value= change.new.getInfo()['id'],
                                   layout = Layout(width = '110px', height = '28px'),
                                   tooltip = change.new.getInfo()['id']
                              )
                
            
            opacity = FloatSlider(
                                      value = 1,
                                      min = 0,  
                                      max = 1,
                                      step = 0.1,
                                      indent=False,   
                               )
            terrain_close_button = close_button
            terrain_visibility  = layer_visibility 
            def change_layer_visibility(show_hide):
                
                layer.visible = show_hide 
                    
            def change_layer_opacity(Opacity):
                layer.opacity = Opacity
                        
            def remove_layer(b):
                map.remove_layer(layer)
                close_button.close()
                opacity.close()
                layer_name.close()
                layer_visibility.close()
                
            opacity_slider = interactive(change_layer_opacity,Opacity = opacity)
                
            visibility_checkbox = interactive(change_layer_visibility,show_hide = layer_visibility)
                
            with TOC_out:
                
                display(HBox([layer_visibility,layer_name,opacity, close_button]))
                   
            close_button.on_click(remove_layer)
terrain.observe(add_terrain , names= 'value')

In [ ]:
map.center = (37.5010, -122.1899)
map

In [ ]:
image = ee.Image('CGIAR/SRTM90_V4')
thumbnail_1 = image.unmask(1000).getThumbURL({
    'min': 0,
    'max': 3000,
    'dimensions': 5000,
})
print('Default extent:', thumbnail_1)



In [ ]:
thumbnail_2 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600', '63C600', 'E6E600', 'E9BD3A', 'ECB176', 'EFC2B3', 'F2F2F2'
    ],
    'dimensions': 500,
    'region': ee.Geometry.Rectangle([-84.6, -55.9, -32.9, 15.7]),
})
print('Rectangle region and palette:', thumbnail_2)

In [ ]:
thumbnail_3 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600', '63C600', 'E6E600', 'E9BD3A', 'ECB176', 'EFC2B3', 'F2F2F2'
    ],
    'region':
        ee.Geometry.LinearRing([[-84.6, 15.7], [-84.6, -55.9], [-32.9, -55.9]]),
    'dimensions': 500,
    'crs': 'EPSG:3857'
})
print('Linear ring region and specified crs:', thumbnail_3)